Digital daydreaming in latent space with CLIP Interogator and Stable Diffusion

In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-730c62ed-159c-3f20-2320-7b012d90b621)


In [ ]:
#@title Installation
import os, subprocess, sys

def setup():
    install_cmds = [
        ['pip', 'install', 'diffusers==0.2.4'],
        ['pip', 'install', 'open_clip_torch', 'ftfy', 'gradio', 'regex', 'tqdm', 'transformers==4.24.0', 'timm', 'fairscale', 'requests'],
        ['pip', 'install', '-e', 'git+https://github.com/MirageML/BLIP.git@main#egg=blip'],#  https://github.com/pharmapsychotic/BLIP.git@lib#egg=blip'],
        ['pip', 'install', 'clip-interrogator==0.3.1' ]
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

# save time of preprocessing by downloading cache from huggingface
print("Download preprocessed cache files...")
CACHE_URLS = [
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_artists.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_flavors.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_mediums.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_movements.pkl',
    'https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_trendings.pkl',
    'https://drive.google.com/file/d/1slcd25W6k-C7RdgTGvQpyrOc1y4A1LyE/view?usp=sharing', #will it work?
]
os.makedirs('cache', exist_ok=True)
for url in CACHE_URLS:
    print(subprocess.run(['wget', url, '-P', 'cache'], stdout=subprocess.PIPE).stdout.decode('utf-8'))

sys.path.append('src/blip')

from clip_interrogator import Config, Interrogator

ci = Interrogator(Config())

def inference(image, mode):
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    else:
        return ci.interrogate_fast(image)


In [3]:
#@title Mount Google Drive
import os
from google.colab import drive

drive.mount('/content/drive')
results_dir = '/content/drive/MyDrive/clip-loop'
os.makedirs(results_dir, exist_ok=True)

# fetch image_to_image.py
!git clone https://github.com/TheOneTrueGuy/DigitalDayDream.git
!cp DigitalDayDream/image_to_image.py /content/

print(f"\nResults will be saved to {results_dir}")
!mkdir vframes
#!cp /content/drive/MyDrive/stub_materials/prlbJulia.zip
!unzip prlbJulia.zip -d vframes

Mounted at /content/drive
Cloning into 'DigitalDayDream'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 59 (delta 26), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (59/59), done.

Results will be saved to /content/drive/MyDrive/clip-loop
cp: missing destination file operand after '/content/drive/MyDrive/stub_materials/prlbJulia.zip'
Try 'cp --help' for more information.
unzip:  cannot find or open prlbJulia.zip, prlbJulia.zip.zip or prlbJulia.zip.ZIP.


In [ ]:
#@title Login to huggingface
!huggingface-cli login

In [ ]:
#https://drive.google.com/file/d/1slcd25W6k-C7RdgTGvQpyrOc1y4A1LyE/view?usp=sharing

In [ ]:
#@title Create animation!

vid_images_path = "/content/vframes/" #@param {type:"string"}
style_prompt = "high contrast cinematic light, mystical shadows, sharp focus" #@param {type:"string"}
dir_name = "frames_out" #@param {type:"string"}
max_frames = 480 #@param {type:"integer"}

import datetime
import os
import torch
import numpy as np
from diffusers import AutoencoderKL
from PIL import Image
from torch import autocast

from image_to_image import StableDiffusionImg2ImgPipeline, preprocess

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)

rond=np.random.randint(7,100000000)
generator = torch.Generator("cuda").manual_seed(rond)
print( "random seed " + str(rond))

device = "cuda"
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    vae=vae,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)
pipe.safety_checker = FakeSafety()

os.makedirs(dir_name, exist_ok=True)
dialog = open("dialog.txt", "a", encoding="utf-8")
dialog.write("random seed:"+str(rond) + "\n")
for xn in range(0, max_frames, 1):
  framename=vid_images_path + str(xn).zfill(4)+".png"
  im = Image.open(framename).convert("RGB")
  im = im.resize((512,512))

  cliprompt = ci.interrogate_fast(im)
  if style_prompt:
    cliprompt = cliprompt.split(", ")
    cliprompt.insert(1, style_prompt)
    cliprompt = ", ".join(cliprompt)
  print(f"{xn:04d}: {cliprompt}\n")  
  dialog.write(f"{xn:04d}: {cliprompt},\n")

  init_image = preprocess(im)
  with autocast("cuda"):
    image = pipe(cliprompt, init_image=init_image, strength=0.68, guidance_scale=6.0, num_inference_steps=20, generator=generator)["sample"][0]  # image
  #image.save("prev.png")
  image.save(f"{dir_name}/{xn:04d}.png")
  # and you will have to compile the video with ffmpeg or something
dialog.close()

# zip animation frames and put in Google Drive
year_month_day = datetime.datetime.now().strftime("%Y_%m_%d")
zip_name = f"{year_month_day}_{dir_name}.zip"

!cp dialog.txt $dir_name/
!zip -r $zip_name $dir_name/ 
!cp $zip_name $results_dir

print(f"Animation frames exported to {results_dir}/{zip_name}")


random seed 1966329


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:00<00:00, 222.89it/s]


0000: a computer generated image of an abstract design, high contrast cinematic light, mystical shadows, sharp focus, fractal bacground, flat surreal psychedelic colors, psychedelic fractal pattern, colorful hyperbolic background, mandelbrot fractal skin, inspired by Benoit B. Mandelbrot, fractal insane, kaleidoscopic, psychedelic fractals, kaleidoscopic colors, fractal style, fractal algorightmic art, fractal render, ai - generated art



0it [00:00, ?it/s]

...

0130: a computer generated image of a psychedelic pattern, high contrast cinematic light, mystical shadows, sharp focus, psychodelic colors, fractal bacground, psychdelic multicolored, psychedelic patterns, mandelbrot fractal skin, kaleidoscopic colors, psychedelic fractal pattern, lsd dream emulator, colorful computer screen, psychadellic colors, bold psychedelic colors, vivid psychedelic colors, flat surreal psychedelic colors



0it [00:00, ?it/s]

100%|██████████| 55/55 [00:00<00:00, 220.57it/s]


0131: a computer generated image of a starfish, high contrast cinematic light, mystical shadows, sharp focus, psychodelic colors, ai - generated art, kaleidoscopic colors, colorful computer screen, psychedelic frog, psychadellic colors, fractal bacground, peyote colors, kaleidoscopic, static colorful noise glitch, lsd dream emulator, an ai generated image, faded lsd colors, acid bleeding deep colors



0it [00:00, ?it/s]

In [ ]:
#run this cell after starting cell above if you don't want to burn 
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()